In [1]:
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
from alexnet import AlexNet
import matplotlib.pyplot as plt
import random
import numpy as np
from sklearn.utils import shuffle
# Visualizations will be shown in the notebook.
%matplotlib inline

In [2]:
#data_dir = '/Users/rajeevkumar/Course/dataset/traffic/'
data_dir = ''
training_file = data_dir + "train.p"
#validation_file= data_dir + "valid.p"
#testing_file = data_dir + "test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
#with open(validation_file, mode='rb') as f:
#    valid = pickle.load(f)
#with open(testing_file, mode='rb') as f:
#    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_train, y_train = shuffle(X_train, y_train)
train_size = int(0.97*len(X_train))

X_valid = X_train[train_size:]
y_valid = y_train[train_size:]
X_train = X_train[:train_size]
y_train = y_train[:train_size]

In [3]:
nb_classes = np.union1d(y_train, y_valid).size

In [4]:
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
resized = tf.image.resize_images(x, (227, 227))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, nb_classes)

In [5]:
fc7 = AlexNet(resized, feature_extract=True)

In [6]:
rate = 0.001
EPOCHS = 10
BATCH_SIZE = 128

In [7]:
# NOTE: `tf.stop_gradient` prevents the gradient from flowing backwards
# past this point, keeping the weights before and up to `fc7` frozen.
# This also makes training faster, less work to do!
fc7 = tf.stop_gradient(fc7)

In [8]:
# TODO: Add the final layer for traffic sign classification.
shape = (fc7.get_shape().as_list()[-1], nb_classes)  # use this shape for the weight matrix
weight_output_layer = tf.Variable(tf.truncated_normal(shape, stddev = 0.01))
bias_output_layer = tf.Variable(tf.zeros([nb_classes]))
traffic_net = tf.add(tf.matmul(fc7, weight_output_layer), bias_output_layer)

In [9]:
with tf.name_scope("xentropy"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=traffic_net)
    loss_operation = tf.reduce_mean(cross_entropy, name="reduced_mean")
    tf.summary.scalar("cross_entroy_mean", loss_operation)

In [10]:
with tf.name_scope("train"):
    #optimizer = tf.train.AdamOptimizer(learning_rate = rate)
    optimizer = tf.train.AdamOptimizer(learning_rate=rate)
    training_operation = optimizer.minimize(loss_operation)

In [11]:
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(traffic_net, 1), tf.argmax(one_hot_y, 1))
    accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy_operation)

In [12]:
def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [13]:
saver = tf.train.Saver()

In [14]:
X_train.shape

(38032, 32, 32, 3)

In [15]:

from sklearn.utils import shuffle
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    writer = tf.summary.FileWriter("./tf/4")
    writer.add_graph(sess.graph)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        train_size = int(0.98*len(X_train))
        X_train_epoch = X_train[: train_size]
        y_train_epoch = y_train[: train_size]
        X_dev_epoch = X_train[train_size:]
        y_dev_epoch = y_train[train_size:]
        for offset in range(0, train_size, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train_epoch[offset:end], y_train_epoch[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
        
        dev_accuracy = evaluate(X_dev_epoch, y_dev_epoch)
        validation_accuracy = evaluate(X_valid, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Dev Accuracy = {:.3f}".format(dev_accuracy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './traffic')
    print("Model saved")

Training...

EPOCH 1 ...
Dev Accuracy = 0.898
Validation Accuracy = 0.895

EPOCH 2 ...
Dev Accuracy = 0.934
Validation Accuracy = 0.934

EPOCH 3 ...
Dev Accuracy = 0.959
Validation Accuracy = 0.950

EPOCH 4 ...
Dev Accuracy = 0.982
Validation Accuracy = 0.954

EPOCH 5 ...
Dev Accuracy = 0.970
Validation Accuracy = 0.964

EPOCH 6 ...
Dev Accuracy = 0.986
Validation Accuracy = 0.963

EPOCH 7 ...
Dev Accuracy = 0.986
Validation Accuracy = 0.970

EPOCH 8 ...
Dev Accuracy = 0.986
Validation Accuracy = 0.973

EPOCH 9 ...
Dev Accuracy = 0.988
Validation Accuracy = 0.973

EPOCH 10 ...
Dev Accuracy = 0.992
Validation Accuracy = 0.975

Model saved
